### Notes

In the simulations, the interleaving methods and AB tests are run and compared counterfactually on the same rankings A and B. They can also be decoupled and run on different synthetic rankings every time.


### Probabilistic interleaving

This class is an adaptation of <a href="https://github.com/HarrieO/PairwisePreferenceMultileave/blob/master/multileaving/ProbabilisticMultileave.py">Harrie Oosterhuis' Probabilistic Multileave implementation</a>, that we customize here to return number of credited clicks rather than binary outcome.

If you use this code or modify its contents in any way, please link back to <a href="https://github.com/HarrieO/PairwisePreferenceMultileave">their repository</a>.

In [ ]:
import numpy as np

class ProbabilisticMultileave(object):

    def __init__(self, nsamples=10000, tau=3.0):
        self.tau = tau
        self.nsamples = nsamples

    def clean(self):
        del self.rankings

    def interleave(self, inverted_rankings):
        '''
        ARGS: (all np.array of docids)
        - inverted_rankings: matrix (rankers x documents) where [x,y] corresponds to the rank of doc y in ranker x

        RETURNS
        - ranking of indices corresponding to inverted_rankings
        '''
        self.rankings = inverted_rankings
        self.nrankers = inverted_rankings.shape[0]
        n = inverted_rankings.shape[1]
        assignments = np.random.randint(0, self.nrankers, n)
        denominator = np.zeros(n) + np.sum(float(1) / (np.arange(n) + 1) ** self.tau)
        probs = 1. / (inverted_rankings[assignments, :] + 1) ** self.tau
        ranking = np.zeros(n, dtype=np.int32)
        docids = np.arange(n)

        for i in range(n):
            upper = np.cumsum(probs[i, :])
            lower = np.zeros(upper.shape)
            lower[1:] += upper[:-1]
            coinflip = np.random.rand()
            logic = np.logical_and(lower / denominator[i] < coinflip, upper / denominator[i] >= coinflip)
            raw_i = np.where(logic)[0][0]
            ranking[i] = docids[raw_i]
            docids[raw_i:-1] = docids[raw_i + 1:]
            denominator -= probs[:, raw_i]
            if raw_i < n - 1:
                probs[:, raw_i:-1] = probs[:, raw_i + 1:]

        return ranking

    def infer_preferences(self, result_list, clicked_docs):
        if np.any(clicked_docs):
            return self.preferences_of_list(self.probability_of_list(result_list,
                                            self.rankings,
                                            clicked_docs.astype(bool), self.tau), self.nsamples)
        else:
            return np.zeros((self.nrankers, self.nrankers))

    def probability_of_list(self, result_list, inverted_rankings, clicked_docs, tau):
        '''
        ARGS: (all np.array of docids)
        - result_list: the multileaved list
        - inverted_rankings: matrix (rankers x documents) where [x,y] corresponds to the rank of doc y in ranker x
        - clicked_docs: boolean array of result_list length indicating clicks

        RETURNS
        -sigmas: matrix (rankers x clicked_docs) with probabilty ranker added clicked doc
        '''
        n_docs = inverted_rankings.shape[1]
        nrankers = inverted_rankings.shape[0]
        click_doc_ind = result_list[clicked_docs]
        # normalization denominator for the complete ranking
        sigmoid_total = np.sum(float(1) / (np.arange(n_docs) + 1) ** self.tau)        
        # cumsum is used to renormalize the probs, it contains the part
        # the denominator that has to be removed due to previously added docs
        cumsum = np.zeros((nrankers, result_list.shape[0]))
        cumsum[:, 1:] = np.cumsum(float(1) / (inverted_rankings[:, result_list[:-1]] + 1.)
                                  ** self.tau, axis=1)
        # make sure inverted rankings is of dtype float
        sigmas = 1 / (inverted_rankings[:, click_doc_ind].T + 1.) ** self.tau
        sigmas /= sigmoid_total - cumsum[:, clicked_docs].T

        return sigmas / np.sum(sigmas, axis=1)[:, None]

    def preferences_of_list(self, probs, nsamples):
        '''
        ARGS:
        -probs: clicked docs x rankers matrix with probabilities ranker added clicked doc (use probability_of_list)
        -nsamples: number of samples to base preference matrix on

        RETURNS:
        - credit vector
        '''
        nclicks = probs.shape[0]
        nrankers = probs.shape[1]
        # determine upper bounds for each ranker (to see prob distribution as set of ranges)
        upper = np.cumsum(probs, axis=1)
        # determine lower bounds
        lower = np.zeros(probs.shape)
        # lower[:,0] = 0
        lower[:, 1:] += upper[:, :-1]
        # flip coins, coins fall between lower and upper
        coinflips = np.random.rand(nclicks, self.nsamples)
        # make copies for each sample and each ranker
        comps = coinflips[:, :, None]
        # determine where each coin landed
        log_assign = np.logical_and(comps > lower[:, None, :], comps <= upper[:, None, :])
        # click count per ranker (samples x rankers)
        click_count = np.sum(log_assign, axis=0)
        # the preference matrix for each sample
        prefs = np.sign(click_count[:, :, None] - click_count[:, None, :])
        
        # the click count is averaged for each pair
        return np.sum(click_count[:, :, None], axis=0) / float(self.nsamples), np.sum(click_count[:, None, :], axis=0) / float(self.nsamples)

### The interleaving methods

In [ ]:
import math
import random

def one(pos):
    return 1

def reciprocal(pos):
    return 1 / (pos+1)

def reciprocal_log(pos):
    return 1 / math.log2(pos+2)

# Slight bias
def clicked(click, view):
    return click+1 if click is not None else None
    
# More bias
def full(act, view):
    return None

# No bias at all
def viewed(act, view):
    return view+1 if view is not None else None
    
class Interleaver:
    def __init__(self, last=viewed, corrected=False):
        self.last = last
        self.corrected = corrected
        
    def reset(self):
        self.ca = self.cb = self.oa = self.ob = self.nqueries = 0
        
    def credit(self, interaction, attribution_a, attribution_b, last_action, last_view):
        last = self.last(last_action, last_view)
        self.ca += sum(map(lambda action, att: action * att, interaction, attribution_a))
        self.cb += sum(map(lambda action, att: action * att, interaction, attribution_b))
        self.oa += sum(attribution_a[0:last])
        self.ob += sum(attribution_b[0:last])
        self.nqueries += 1

    def session_credit(self):
        if self.corrected:
            return self.ca / self.oa * (self.oa + self.ob) / self.nqueries if self.oa > 0 else 0, \
                   self.cb / self.ob * (self.oa + self.ob) / self.nqueries if self.ob > 0 else 0
        else:
            return self.ca / self.nqueries, self.cb / self.nqueries
        
    def interact(self, customer, I, rel):
        self.interaction, self.last_action, self.last_view = customer.interact(I, rel)
        return self.interaction, self.last_action, self.last_view

class Balanced(Interleaver):
    def __init__(self, last=viewed, corrected=False, clone=False):
        super().__init__(last, corrected)
        self.clone = clone

    def __repr__(self):
        name = 'Corrected balanced' if self.corrected else 'Uncorrected balanced'
        return name

    def interleave(self, A, B):
        if self.clone: return self.clone.I
        self.I = []
        self.attribution_a = []
        self.attribution_b = []
        toss = int(random.random() < 0.5)
        first, second = (A, B) if toss else (B, A)
        for x, y in zip(first, second):
            if x == y: 
                self.I.append(x)
                self.attribution_a.append(0)
                self.attribution_b.append(0)
            else:
                if not x in self.I: 
                    self.I.append(x)
                    self.attribution_a.append(toss)
                    self.attribution_b.append(1-toss)
                if not y in self.I: 
                    self.I.append(y)
                    self.attribution_a.append(1-toss)
                    self.attribution_b.append(toss)
        return self.I

    def attribution(self, I, A, B, last_action=None, last_view=None):
        if self.clone: return self.clone.attribution_a, self.clone.attribution_b
        else: return self.attribution_a, self.attribution_b

    def interact(self, customer, I, rel):
        if self.clone: return self.clone.interaction, self.clone.last_action, self.clone.last_view 
        else: return super().interact(customer, I, rel)
        
class WeightedBalanced(Balanced):
    def __init__(self, weight=one, last=viewed, corrected=False):
        super().__init__(last, corrected)
        self.weight = weight

    def __repr__(self):
        return 'Corrected weighted balanced' if self.corrected else 'Weighted balanced' 

    def attribution(self, I, A, B, last_action=None, last_view=None):
        attribution_a = [self.weight(A.index(x)) if x in A else 0 for x in I]
        attribution_b = [self.weight(B.index(x)) if x in B else 0 for x in I]
        return attribution_a, attribution_b

class OriginalBalanced(WeightedBalanced):
    def __init__(self, weight=one, last=viewed, corrected=False):
        super().__init__(weight, last, corrected)

    def __repr__(self):
        return 'Corrected original balanced' if self.corrected else 'Orignal balanced' 

    def attribution(self, I, A, B, last_action=None, last_view=None):
        x = I[last_action]
        last = min(A.index(x) if x in A else last_action, B.index(x) if x in B else last_action)
        A, B = A[0:last+1], B[0:last+1]
        attribution_a = [self.weight(A.index(x)) if x in A else 0 for x in I]
        attribution_b = [self.weight(B.index(x)) if x in B else 0 for x in I]
        return attribution_a, attribution_b

class TeamDraft(Interleaver):
    def __init__(self, last=viewed, corrected=False):
        super().__init__(last, corrected)
    def __repr__(self):
        return 'Team draft' 

    def interleave(self, A, B):
        I = []
        self.attribution_a = []
        self.attribution_b = []
        inI = [0] * (len(A)+1) # trick for speedup, assumes A and B contain items 0, 1, ..., len(A)
        i = j = 0
        na = nb = 0
        while True:
            Afirst = int(na < nb) or (int(na == nb) and random.randint(0,1))
            if Afirst: 
                i = min([k for k in range(i, len(A)) if not inI[A[k]]] + [len(A)])
                na += 1
                R, k = A, i
            else:
                j = min([k for k in range(j, len(B)) if not inI[B[k]]] + [len(B)])
                nb += 1
                R, k = B, j
            if k == len(R): break
            I.append(R[k])
            inI[R[k]] = 1
            self.attribution_a.append(Afirst)
            self.attribution_b.append(1-Afirst)
        return I

    def attribution(self, I, A, B, last_action=None, last_view=None):
        return self.attribution_a, self.attribution_b

class Probabilistic(TeamDraft):
    def __init__(self, last=viewed, corrected=False, nsamples=1000):
        super().__init__(last, corrected)
        self.multileaver = ProbabilisticMultileave(nsamples=nsamples)

    def __repr__(self):
        return 'Probabilistic' 

    def interleave(self, A, B):
        rankings = np.array([invert(A), invert(B)])
        I = self.multileaver.interleave(rankings)
        att = self.multileaver.probability_of_list(I, rankings, np.repeat(True, len(I)), self.multileaver.tau)
        self.attribution_a, self.attribution_b = att[:, 0].tolist(), att[:, 1].tolist()
        return I

class Prob(Probabilistic):
    def __init__(self, last=viewed, corrected=False, nsamples=1000):
        super().__init__(last, corrected)
        self.multileaver = ProbabilisticMultileave(nsamples=nsamples)
        self.attribution_a = self.attribution_b = 0

    def __repr__(self):
        return 'Prob ' + str(self.multileaver.nsamples)

    def interleave(self, A, B):
        rankings = np.array([invert(A), invert(B)])
        self.I = self.multileaver.interleave(rankings)
        return self.I

    def credit(self, interaction, attribution_a, attribution_b, last_action, last_view):
        if (sum(interaction) > 0):
            ca, cb = tuple(self.multileaver.infer_preferences(self.I[0:len(interaction)], np.array(interaction)))[0]
            self.ca += ca[0]
            self.cb += cb[0]
        self.nqueries += 1

def invert(ranking):
    r = [0] * len(ranking)
    for i in range(len(ranking)): r[ranking[i]] = i
    return r



### Customer model

In [ ]:
import math

def ndcg_stop(k):
    return 1 - math.log(k+2) / math.log(k+3)

class CustomerModel():
    def __init__(self, type, pact_rel, pact_nonrel, pstop_rel, pstop_nonrel):
        self.type, self.pact_rel, self.pact_nonrel, self.pstop_rel, self.pstop_nonrel = type, pact_rel, pact_nonrel, pstop_rel, pstop_nonrel

    def __repr__(self):
        return self.type

    def act(self, a, rel, k):
        return int(random.random() < (self.pact_rel if a == rel else self.pact_nonrel))

    def stop(self, a, rel, k):
        return random.random() < (self.pstop_rel(k) if a == rel else self.pstop_nonrel(k))

    def interact(self, ranking, rel):
        interaction = []
        last = -1
        for pos, x in enumerate(ranking):
            action = self.act(x, rel, pos)
            interaction.append(action)
            if action: last = pos
            if self.stop(x, rel, pos): break
        return interaction, last, pos

### Simulation of rankings and sessions

Also includes routines that compute experiment power for a range of sample sizes (number of sessions), plots of interleaved vs. AB mean difference between a pair of rankers, effect size and covariance.

In [1]:
import numpy as np
import scipy.stats as st
import math

def td_edge(nonrel, rel):
    return [1, 2, rel], [2, rel, 1]

def ranking(nonrel, rel, posA, posB):
    A = nonrel.copy()
    B = nonrel.copy()
    A.insert(posA,rel)
    B.insert(posB,rel)
    return A, B

# A has one relevant ASIN at a random position, and B has it one position higher.
def b_one_higher(nonrel, rel):
    pos = random.randint(1, len(nonrel))
    return ranking(nonrel, rel, pos, pos-1)

def b_above(nonrel, rel):
    posA = random.randint(0, len(nonrel)+1)
    posB = random.randint(round(posA/2), posA+1)
    return ranking(nonrel, rel, posA, posB)

def b_closer(nonrel, rel):
    posA = random.randint(0, math.ceil((len(nonrel)+1)/2))
    posB = random.randint(0, posA+1)
    return ranking(nonrel, rel, posA, posB)

def b_higher(nonrel, rel):
    n = len(nonrel)+1
    k = round(n/2)
    return ranking(nonrel, rel, random.randint(k, n+1), random.randint(0, k))

# A has one relevant ASIN at the last position, and B has it at a random higher position.
def a_last(nonrel, rel):
    pos = random.randint(0, round(len(nonrel)/2)-1)
    return ranking(nonrel, rel, pos, len(nonrel))

def run(nsessions, nqueries, nonrel, rel, ranking_generator, customer, interleavers):
    absessions = math.ceil(nsessions/2)
    a = [0] * absessions
    b = [0] * absessions
    ia = [[0] * nsessions for i in interleavers]
    ib = [[0] * nsessions for i in interleavers]
    for session in range(nsessions):
        for interleaver in interleavers: interleaver.reset()
        for _ in range(nqueries):
            A, B = ranking_generator(nonrel, rel)
            if session % 2 == 0:
                a[int(session/2)] += sum(customer.interact(A, rel)[0])
                b[int(session/2)] += sum(customer.interact(B, rel)[0])
            # A, B = ranking_generator(nonrel, rel)
            for interleaver in interleavers:
                I = interleaver.interleave(A, B)
                interaction, last_action, last_view = interleaver.interact(customer, I, rel)
                attribution = interleaver.attribution(I, A, B, last_action, last_view)
                interleaver.credit(interaction, *attribution, last_action, last_view)
        if session % 2 == 0:
            a[int(session/2)] /= nqueries
            b[int(session/2)] /= nqueries
        for i, interleaver in enumerate(interleavers):
            ia[i][session], ib[i][session] = interleaver.session_credit()
    return a, b, ia, ib

def power(alpha, npoints, nreps, nsessions, nqueries, nonrel, rel, ranking_generator, customer, interleavers):
    print("Power " + str(len(interleavers)) + " interleaving methods (" + str(customer) + " customer) x " + str(nsessions) + " sessions x " + str(nqueries) + " queries x " + str(nreps) + " reps -", end=' ')
    abpower = [0] * npoints
    ipower = [[0] * npoints for i in interleavers]
    x = [0] * npoints
    for point in range(npoints):
        nsignificant = 0
        size = nsessions * 10**(-2 + 2 * point / (npoints-1))
        x[point] = size / nsessions
        for i in range(nreps):
            a, b, ia, ib = run(round(size), nqueries, nonrel, rel, ranking_generator, customer, interleavers)
            abpower[point] += int(st.ttest_ind(a, b).pvalue < alpha)
            for ip, a, b in zip(ipower, ia, ib):
                ip[point] += st.ttest_rel(a, b).pvalue < alpha
        abpower[point] /= nreps
        for ip in ipower: ip[point] /= nreps
        print(str(round(size)) + "/" + str(nsessions), end=' ')
    print()
    result = [{'method':str(interleaver), 'x':x, 'power':ip} for interleaver, ip in zip(interleavers, ipower)]
    result.append({'method':"A/B test", 'x':x, 'power':abpower})
    return result

def correlation(npoints, nsessions, nqueries, nonrel, rel, ranking_generator, interleavers):
    print("Correlation AB / " + str(len(interleavers)) + " interleaving methods x " + str(nsessions) + " sessions x " + str(nqueries) + " queries")
    pstop_rel = ndcg_stop
    pstop_nonrel = ndcg_stop
    ab = []
    results = []
    for interleaver in interleavers: results.append({'method':str(interleaver), 'inter':[]})
    n = round((npoints+1)*npoints/2)
    k = 0
    for i in range(1, npoints):
        for j in range(i, npoints+1):
            k += 1
            print(str(k) + "/"+ str(n), end=' ')
            pact_nonrel, pact_rel = i / npoints, j / npoints
            customer = CustomerModel("non-uniform", pact_rel, pact_nonrel, pstop_rel, pstop_nonrel)
            a, b, ia, ib = run(nsessions, nqueries, nonrel, rel, ranking_generator, customer, interleavers)
            flip = 1 if random.random() < .5 else -1
            ab.append((flip*(np.mean(b)-np.mean(a)), st.ttest_ind(a, b).pvalue))
            for result, ca, cb in zip(results, ia, ib): 
                result['inter'].append((flip*(np.mean(cb)-np.mean(ca)), st.ttest_rel(ca, cb).pvalue))
    print()
    return ab, results

### Convenience functions for running sets of tests

Run combinations of interleaved methods and uniform vs. informed customer behavior for different measurements: raw values, experiment power, and effect size.

In [ ]:
import threading

def make_dic(interleaver, a, b):
    return {'method': str(interleaver), 'a':a, 'b':b}

def make_abdic(a, b):
    return make_dic('A/B test', a, b)

def values_all(nsessions, nqueries, nonrel, rel, ranking_generator, customers, interleavers):
    all_values = []
    for customer in customers:
        print("Running " + str(len(interleavers)) + " interleaving methods (" + str(customer) + " customer) x " + str(nsessions) + " sessions x " + str(nqueries) + " queries...")
        a, b, ia, ib = run(nsessions, nqueries, nonrel, rel, ranking_generator, customer, interleavers)
        results = [make_dic(interleaver, aa, bb) for interleaver, aa, bb in zip(interleavers, ia, ib)]
        results.append(make_abdic(a, b))
        all_values.append(results)
    return tuple(all_values)

def power_all(alpha, npoints, nreps, nsessions, nqueries, nonrel, rel, ranking_generator, customers, interleavers):
    return tuple([power(alpha, npoints, nreps, nsessions, nqueries, nonrel, rel, ranking_generator, customer, interleavers) for customer in customers])

### Convenience functions for plotting batches of results

In [ ]:
from matplotlib import ticker, pyplot as pt
import numpy as np
import scipy

def plot1(tied_results, results, tied_presults, presults, filename, labels=False,
               markers=['o', 'o', 'x', '^', '+', '+', '1'], sizes=[8, 8, 8, 10, 10, 10, 10],
               colors=['blue', 'red', 'blue', 'green', 'darkorange', 'red', 'black']):
    if not labels: labels = [series['method'] for series in tied_results]
    fig, ((tied_scatter, scatter), (tied_power, power)) = pt.subplots(2, 2, dpi=300)
    fig.set_size_inches(6.1, 6.25, forward=True)
    pt.subplots_adjust(wspace=0.33, hspace=.28)
        
    # Common bounds for value graphs
    mn = min(0, min((min(*series['a'], *series['b']) for series in results + tied_results)))
    mx = max(max(*series['a'], *series['b']) for series in results + tied_results)
    mn, mx = mn - mx / 20, mx + mx / 20

    tied_scatter.set_xlim(mn, mx)
    tied_scatter.set_ylim(mn, mx)
    scatter.set_xlim(mn, mx)
    scatter.set_ylim(mn, mx)
    # y = x line
    tied_scatter.plot([mn, mx], [mn, mx], color='k', linewidth=0.5, linestyle='--', mec='none', markerfacecolor='None')
    tied_scatter.text(mx*.03, mx*.08, r"$y=x$", rotation=45, rotation_mode='anchor')
    scatter.plot([mn, mx], [mn, mx], color='k', linewidth=0.5, linestyle='--', mec='none', markerfacecolor='None')
    scatter.text(mx*.03, mx*.08, r"$y=x$", rotation=45, rotation_mode='anchor')

    tied_scatter.set_title("Random user behavior")
    tied_scatter.set_xlabel(r"Credit$\,(A)$")
    tied_scatter.set_ylabel(r"Credit$\,(B)$")
    tied_scatter.set_xticks([0, 2, 4, 6, 8])
    tied_scatter.set_yticks([0, 2, 4, 6, 8])
    scatter.set_title("Non-random user behavior")
    scatter.set_xlabel(r"Credit$\,(A)$")
    scatter.set_xticks([0, 2, 4, 6, 8])
    scatter.set_yticks([0, 2, 4, 6, 8])
    tied_power.set_ylabel(r"$P\,(p$-value$<0.05)$")
    tied_power.set_ylim(-.05, 1.05)
    tied_power.set_xlabel("Search traffic sample size\n(ratio in # sessions)")
    tied_power.set_xscale('log')
    tied_power.set_xticks(tied_presults[0]['x'])
    tied_power.grid(True, linewidth=.25, color='dimgray')
    power.set_xlabel("Search traffic sample size\n(ratio in # sessions)")
    power.set_xscale('log')
    power.set_xticks(presults[0]['x'])
    power.grid(True, linewidth=.25, color='dimgray')
    markeredgewidth = .5

    ## Tied results

    for series, marker, markersize, color, method in zip(tied_results, markers, sizes, colors, labels):
        tied_scatter.plot(series['a'], series['b'], label=method, linestyle='None',  marker=marker, color=color, linewidth=.6, markersize=markersize, markerfacecolor='None', markeredgewidth=markeredgewidth)

    handles, labels = tied_scatter.get_legend_handles_labels()
    handles[0], handles[1] = handles[1], handles[0]
    labels[0], labels[1] = labels[1], labels[0]
    tied_scatter.legend(handles, labels, bbox_to_anchor=(2.37, -1.58), ncol=3, columnspacing=.8, handletextpad=.3, labelspacing=.32)

    for series, marker, markersize, color, method in zip(tied_presults, markers, sizes, colors, labels):
        tied_power.plot(series['x'], series['power'], label=method, marker=marker, color=color, linewidth=.6, markersize=markersize, markerfacecolor='white', markeredgewidth=.8)

    ## Non-tied results

    for series, marker, markersize, color, method in zip(results, markers, sizes, colors, labels):
        scatter.plot(series['a'], series['b'], label=method, linestyle='None', marker=marker, color=color, linewidth=.6, markersize=markersize, markerfacecolor='None', markeredgewidth=markeredgewidth)
    
    for series, marker, markersize, color, method in zip(presults, markers, sizes, colors, labels):
        power.plot(series['x'], series['power'], label=method, marker=marker, color=color, linewidth=.6, markersize=markersize, markerfacecolor='white', markeredgewidth=.8)

    pt.savefig(filename, bbox_inches='tight')

def plot2(results, presults, filename, labels=False,
               markers=['o', 'o', 'x', '^', '+', '+', '1'], sizes=[8, 8, 8, 10, 10, 10, 10],
               colors=['blue', 'red', 'blue', 'green', 'darkorange', 'red', 'black']):
    if not labels: labels = [series['method'] for series in tied_results]
    fig, (scatter, power) = pt.subplots(1, 2, dpi=300)
    fig.set_size_inches(6, 2.65, forward=True)
    pt.subplots_adjust(wspace=.33)

    # Common bounds for value graphs
    mn = min(0, min((min(*series['a'], *series['b']) for series in results)))
    mx = max(max(*series['a'], *series['b']) for series in results)
    mn, mx = mn - mx / 20, mx + mx / 20

    scatter.set_xlim(mn, mx)
    scatter.set_ylim(mn, mx)
    # y = x line
    scatter.plot([mn, mx], [mn, mx], color='k', linewidth=.5, linestyle='--', mec='none', markerfacecolor='None')
    bbox = scatter.get_window_extent().transformed(fig.dpi_scale_trans.inverted())
    angle = np.arctan(bbox.height/bbox.width) / math.pi * 180
    scatter.text(mx*.03, mx*.08, r"$y=x$", rotation=angle, rotation_mode='anchor')
    scatter.set_ylabel(r"Credit$\,(B)$")
    scatter.set_xlabel(r"Credit$\,(A)$")

    power.set_ylim(-.05, 1.05)
    power.set_ylabel(r"$P\,(p$-value$<0.05)$")
    power.set_xlabel("Search traffic sample size\n(ratio in # sessions)")
    power.set_xscale('log')
    power.set_xticks(presults[0]['x'])
    power.grid(True, linewidth=.25, color='dimgray')
    markeredgewidth = .5

    for series, marker, markersize, color, method in zip(results, markers, sizes, colors, labels):
        scatter.plot(series['a'], series['b'], label=method, linestyle='None', marker=marker, color=color, linewidth=.6, markersize=markersize, markerfacecolor='None', markeredgewidth=markeredgewidth)
    scatter.legend(bbox_to_anchor=(2.37, -.3), 
                       ncol=4, 
                       columnspacing=.8, 
                       handletextpad=.28 
                      )
    
    for series, marker, markersize, color, method in zip(presults, markers, sizes, colors, labels):
        power.plot(series['x'], series['power'], label=method, marker=marker, color=color, linewidth=.6, markersize=markersize, markerfacecolor='white', markeredgewidth=.8)

    pt.savefig(filename, bbox_inches='tight')

# Two power graphs
def plot3(presults1, presults2, labels1, labels2, filename):
    fig, (power1, power2) = pt.subplots(1, 2, dpi=300)
    fig.set_size_inches(6, 2.7, forward=True)
    pt.subplots_adjust(wspace=0.3)
    
    power1.set_title("a)  Random user behavior")
    power1.set_ylim(-.05, 1.05)
    power1.set_ylabel(r"$P\,(p$-value$<0.05)$")
    power1.set_xlabel("Search traffic sample size\n(ratio in # sessions)")
    power1.set_xscale('log')
    power1.set_xticks(tied_presults[0]['x'])
    power1.grid(True, linewidth=.25, color='dimgray')
    
    power2.set_title("b)  Non-random user behavior   ")
    power2.set_ylim(-.05, 1.05)
    power2.set_xlabel("Search traffic sample size\n(ratio in # sessions)")
    power2.set_xscale('log')
    power2.set_xticks(presults[0]['x'])
    power2.grid(True, linewidth=.25, color='dimgray')
    markeredgewidth = .5

    for series, marker, markersize, color, method in zip(presults1, ['o', 'x', '^'], \
                                                 [8, 8, 8], \
                                                 ['red', 'blue','green'], \
                                                 labels1):
        power1.plot(series['x'], series['power'], label=method, marker=marker, color=color, linewidth=.6, markersize=markersize, markerfacecolor='white', markeredgewidth=.8)

    for series, marker, markersize, color, method in zip(presults2, ['o', 'x', '^'], \
                                                 [8, 8, 8], \
                                                 ['red', 'blue','green'], \
                                                 labels2):
        power2.plot(series['x'], series['power'], label=method, marker=marker, color=color, linewidth=.6, markersize=markersize, markerfacecolor='white', markeredgewidth=.8)
    power2.legend(bbox_to_anchor=(-.38, .22, .5, .5), framealpha=.95)

    pt.savefig(filename, bbox_inches='tight')
    
def plotcorrelation(ab, results, labels, filename):
    fig, plots = pt.subplots(1, len(results), dpi=300)
    fig.set_size_inches(2.9 * len(results), 2.35, forward=True)
    pt.subplots_adjust(wspace=0.27)
    plot_row(plots, ab, results, labels, ['o', 'o', 'x', '^', '+', 'x'], [6, 6, 6, 8, 8, 6], ['red', 'blue', 'blue', 'green', 'darkorange', 'purple'])
    pt.savefig(filename, bbox_inches='tight')

def plot_row(plots, ab, results, labels, markers, sizes, colors):
    x = [x[0] for x in ab]
    plots[0].set_ylabel("Interleaved\ncredit$\,(B)$ $-$ credit$\,(A)$", fontsize=12)
    markeredgewidth = .5
    for p, r, method, marker, size, color in zip(plots, results, labels, markers, sizes, colors):
        x_sig = [x[0] for x, y in zip(ab, r['inter']) if y[1] < .05]
        x_nosig = [x[0] for x, y in zip(ab, r['inter']) if y[1] >= .05]
        y = [y[0] for y in r['inter']]
        y_sig = [y[0] for x, y in zip(ab, r['inter']) if y[1] < .05]
        y_nosig = [y[0] for x, y in zip(ab, r['inter']) if y[1] >= .05]
        p.axis('on')
        p.set_title(method, fontsize=13)
        p.set_xlabel("credit$\,(B)$ $-$ credit$\,(A)$\nA/B test", fontsize=12)
        mnx, mxx = min(x_sig), max(x_sig)
        mnx, mxx = min(mnx, -mxx), max(-mnx, mxx)
        mny, mxy = min(y_sig), max(y_sig)
        mny, mxy = min(mny, -mxy), max(-mny, mxy)
        w, h = (mxx - mnx) / 20, (mxy - mny) / 20
        exponent = np.floor(np.log10(mxy))
        scale = 10**-exponent if exponent < -1 else 1
        if abs(mxy - .1) < .02: scale = 10
        mnx, mxx = mnx - w, mxx + w
        mny, mxy = mny - h, mxy + h
        p.set_xlim(mnx, mxx)
        p.set_ylim(mny * scale, mxy * scale)
        p.plot([0, 0], [mny * scale, mxy * scale], color='k', linewidth=.5, mec='none', markerfacecolor='None')
        p.plot([mnx, mxx], [0, 0], color='k', linewidth=.5, mec='none', markerfacecolor='None')
        p.ticklabel_format(axis='y', style='plain')
        if scale > 1: p.annotate(r'$\times$10$^{%i}$'%(exponent), xy=(.01, 0.9), xycoords='axes fraction')

        p.plot(x, [y * scale for y in y], label=method, linestyle='None', marker=marker, color=color, linewidth=.6, markersize=size, markeredgewidth=markeredgewidth, markerfacecolor='None')
            
        slope, _ = scipy.optimize.curve_fit(lambda x, slope: slope*x, x, y, method='dogbox')
        p.plot([mnx, mxx], [slope * mnx * scale, slope * mxx * scale], color='k', linewidth=.5, linestyle='--', mec='none')
        corr, _ = scipy.stats.pearsonr(x, y)
        p.text(.95, .13, r"$r=" + str(round(corr, 2)) + "$", 
               horizontalalignment='right', verticalalignment='top', transform = p.transAxes,
               fontsize=12, bbox=dict(facecolor='white', alpha=0.7, edgecolor='None'))

-------
## Top-level experiment calls and plotting

### 0.  Common settings

In [ ]:
nsessions = 100
nqueries = 100
pact_rel = 1
pact_nonrel = .5
pstop_rel = ndcg_stop
pstop_nonrel = ndcg_stop
nonrel = list(range(1,50))       # Non relevant ASINs
rel = 0                          # One relevant ASIN
# ranking_generator = a_last
# ranking_generator = b_one_higher 
# ranking_generaçtor = b_above
ranking_generator = b_higher
# ranking_generator = td_edge
customer = CustomerModel("non-uniform", pact_rel, pact_nonrel, pstop_rel, pstop_nonrel)
uniform_customer = CustomerModel("uniform", .5, pact_nonrel, pstop_rel, pstop_nonrel)
customers = [uniform_customer, customer]
alpha = .05
nreps = 1000

### 1.  First fig in simulation section: correction reliability and power

In [ ]:
import time

npoints = 9

interleavers = [Balanced(), Balanced(last=viewed, corrected=True), OriginalBalanced(), WeightedBalanced(reciprocal_log), TeamDraft(), Prob(nsamples=100)]
ranking_generator = b_closer

t = time.time()
tied_results, results = values_all(nsessions, nqueries, nonrel, rel, ranking_generator, customers, interleavers)
t = round(time.time() - t)
print(round(t/60), 'min', t % 60, 'sec')

t = time.time()
tied_presults, presults = power_all(alpha, npoints, nreps, nsessions, nqueries, nonrel, rel, ranking_generator, customers, interleavers)
t = round(time.time() - t)
print(round(t/60), 'min', t % 60, 'sec')

labels = ['Uncorrected balanced', 'Debiased balanced', 'Original balanced', 'Weighted balanced', 'Team draft', 'Probabilistic', 'A/B test']
plot1(tied_results, results, tied_presults, presults, 'simulation-' + str(nsessions) + 'x' + str(nqueries) + 'x' + str(nreps) + '.pdf', labels=labels,
           markers=['o', 'o', 'x', '^', '+', 'x', '1'], sizes=[8, 8, 8, 10, 10, 8, 10],
           colors=['blue', 'red', 'blue', 'green', 'darkorange', 'purple', 'black'])

### 2. Second fig in simulation: team draft edge case

In [ ]:
import time

npoints = 9

interleavers = [Balanced(last=viewed, corrected=True), TeamDraft(), Prob(nsamples=100)]
ranking_generator = td_edge

t = time.time()
tde_results = values_all(nsessions, nqueries, nonrel, rel, ranking_generator, [customer], interleavers)[0]
t = round(time.time() - t)
print(round(t/60), 'min', t % 60, 'sec')

t = time.time()
tde_presults = power_all(alpha, npoints, nreps, nsessions, nqueries, nonrel, rel, ranking_generator, [customer], interleavers)[0]
t = round(time.time() - t)
print(round(t/60), 'min', t % 60, 'sec')

labels = ['Debiased balanced', 'Team draft', 'Probabilistic', 'A/B test']
plot2(tde_results, tde_presults, 'tdedge-' + str(nsessions) + 'x' + str(nqueries) + 'x' + str(nreps) + '.pdf', labels=labels,
           markers=['o', '+', 'x', '1'], sizes=[8, 10, 8, 10],
           colors=['red', 'darkorange', 'purple', 'black'])

for res in results: print(res['method'], np.mean(res['a'])-np.mean(res['b']))

### 3. Third fig in simulation section: customer opportunity and other variants

In [ ]:
interleavers = [Balanced(corrected=True, last=clicked), Balanced(corrected=True, last=full)]
ranking_generator = b_higher

interleavers = [OriginalBalanced(corrected=True), WeightedBalanced(reciprocal_log, corrected=True)]
tied_alt_presults, alt_presults = power_all(alpha, npoints, nreps, nsessions, nqueries, nonrel, rel, ranking_generator, [uniform_customer, customer], interleavers)
tied_alt_presults = [tied_presults[1]] + tied_alt_presults[:-1]
alt_presults = [presults[1]] + alt_presults[:-1]

alt_labels = ['Our attribution policy', 'Original policy', 'Weighted policy']
pplot3(tied_alt_presults, alt_presults, alt_labels, alt_labels, 'variations-' + str(nsessions) + 'x' + str(nqueries) + 'x' + str(nreps) + '.pdf')

### 4.  Fourth fig in simulation section: correlation

In [ ]:
import time

interleavers = [Balanced(), Balanced(corrected=True), OriginalBalanced(), WeightedBalanced(reciprocal_log), TeamDraft(), Prob(nsamples=100)]
ranking_generator = b_higher

t = time.time()
npoints = 10
nsessions = 10000
ab_result, il_results = correlation(npoints, nsessions, nqueries, nonrel, rel, ranking_generator, interleavers)
t = round(time.time() - t)
print(round(t/60), 'min', t % 60, 'sec')

labels = ['Debiased balanced', 'Uncorrected balanced', 'Original balanced', 'Weighted balanced', 'Team draft', 'Probabilistic']
plotcorrelation(ab_result, il_results, labels, 'correlation-' + str(nsessions) + 'x' + str(nqueries) + '.pdf')